#  Introduction
Here, the code for the research project "Modelling Options Prices Considering Mandelbrotian Movement of Prices" is located. The comments are provided in the code for better readability.

The entire code of the research project can be divided into the following four parts: 
1) Models classes (Classical Black-Scholes, Fractional Black-Scholes, Multifractional Black-Scholes, Merton models). 
2) Reading and preparation of datasets. The dataset (options data, risk-free rate data) is required for further parameter optimization and backtesting. 
3) Backtesting or Simulation. This part involves k-fold cross-validation for each trading day in the 2020-2022 period, optimization of model parameters on training data, and calculation of options price estimates on test data. 
4) Metrics evaluation and visualization. 

**Contents**
- [Introduction](#Introduction)
- [Models section](#Models-section)
- [Market options data](#Market-options-data)
- [S&P 500 spot and risk-free rate data](#S&P-500-spot-and-risk-free-rate-data)
- [Simulation](#Simulation)
- [Performance evaluation](#Performance-evaluation)

In [6]:
import pandas as pd
import numpy as np
from arch import arch_model
import yfinance as yf
import time
import math
from tqdm import tqdm
from datetime import date, datetime, timedelta
import os
from scipy.stats import norm
from scipy.optimize import minimize
from arch.__future__ import reindexing
import copy
from sklearn.model_selection import cross_val_predict

# Models section

In [7]:

# Black-Scholes option prices
def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + r*T + 0.5*(sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r*T)* norm.cdf(d2)

def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

In [8]:
import numpy as np

# MERTON MODEL
class Merton:
    def __init__(self, S=None, K=None, r=None, sigma=None, m=None, v=None, lam=None, x0=[0.1], bounds=[(0.01, 0.9)]):
        self.sigma = sigma
        self.r = r
        self.S = S
        self.K = K
        self.m = m
        self.v = v
        self.lam = lam
        self.x0 = x0
        self.bounds=bounds

    def predict(self, X):
        return X.apply(lambda row: self.call(self.S, self.K, row['dte']/365.0, self.r, self.sigma, self.m, self.v, self.lam), axis=1).tolist()

    def call(self, S, K, T, r, sigma, m, v, lam):
        p = 0
        for k in range(20):
            r_k = r - lam*(m-1) + (k*np.log(m) ) / T
            sigma_k = np.sqrt(sigma**2 + (k* v** 2) / T)
            k_fact = np.math.factorial(k)
            p += (np.exp(-m*lam*T) * (m*lam*T)**k / (k_fact))  * BS_CALL(S, K, T, r_k, sigma_k)
        
        return p 

    def opt(self, x, X, y):
        candidate_prices = X.apply(lambda row: self.call(self.S, self.K, row['dte']/365.0, self.r, x[0], self.m, self.v, self.lam), axis=1).tolist()
        return np.linalg.norm(y - candidate_prices, 2)

    def fit(self, X, y, **kwargs):
        self.r = kwargs['r'][0]
        self.S = kwargs['S'][0]
        self.K = kwargs['K'][0]
        self.m = 0.1
        self.v = 0.1
        self.lam = 0.01
        res = minimize(self.opt, method='SLSQP',  x0=self.x0, args=(X, y),
                bounds=self.bounds, tol=1e-10, 
                options={"maxiter":1000})
        self.sigma = res.x[0]
        return self
    
    def get_params(self, deep = False):
        return {'sigma':self.sigma}

# CLASSICAL BLACK-SCHOLES
class BlackScholes:
    def __init__(self, S=None, K=None, r=None, sigma=None, x0=0.1, bounds=[(0.01, 0.9)]):
        self.sigma = sigma
        self.r = r
        self.S = S
        self.K = K
        self.x0 = x0
        self.bounds=bounds

    def predict(self, X):
        return X.apply(lambda row: self.call(self.S, self.K, row['dte']/365.0, self.r, self.sigma), axis=1).tolist()

    def call(self, S, K, T, r, sigma):
        d1 = (np.log(S/K) + r*T + 0.5*(sigma**2)*T) / (sigma*np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return S * norm.cdf(d1) - K * np.exp(-r*T)* norm.cdf(d2)

    def opt(self, x, X, y):
        candidate_prices = X.apply(lambda row: self.call(self.S, self.K, row['dte']/365.0, self.r, x[0]), axis=1).tolist()
        return np.linalg.norm(y - candidate_prices, 2)

    def fit(self, X, y, **kwargs):
        self.r = kwargs['r'][0]
        self.S = kwargs['S'][0]
        self.K = kwargs['K'][0]
        res = minimize(self.opt, method='SLSQP',  x0=self.x0, args=(X, y),
                bounds=self.bounds, tol=1e-10, 
                options={"maxiter":1000})
        self.sigma = res.x[0]
        return self
    
    def get_params(self, deep = False):
        return {'sigma':self.sigma}

#FRACTIONAL BLACK-SCHOLES
class FractionalBlackScholes:
    def __init__(self, S=None, K=None, r=None, sigma=None, x0=[0.1, 0.5], bounds=[(0.01, 0.9), (0.01, 0.99)], H=0.5):
        self.sigma = sigma
        self.r = r
        self.S = S
        self.K = K
        self.H = H
        self.x0 = x0
        self.bounds=bounds

    def predict(self, X):
        return X.apply(lambda row: self.call(self.S, self.K, row['dte']/365.0, self.r, self.sigma, self.H), axis=1).tolist()

    def call(self, S, K, T, r, sigma, H):
        d1 = (np.log(S/K) + r*T + 0.5*(sigma**2)*T**(2*H)) / (sigma*np.sqrt(T**(2*H)))
        d2 = d1 - sigma * np.sqrt(T**(2*H))
        return S * norm.cdf(d1) - K * np.exp(-r*T)* norm.cdf(d2)

    def opt(self, x, X, y):
        candidate_prices = X.apply(lambda row: self.call(self.S, self.K, row['dte']/365.0, self.r, x[0], x[1]), axis=1).tolist()
        return np.linalg.norm(y - candidate_prices, 2)

    def fit(self, X, y, **kwargs):
        self.r = kwargs['r'][0]
        self.S = kwargs['S'][0]
        self.K = kwargs['K'][0]
        res = minimize(self.opt, method='SLSQP',  x0=self.x0, args=(X, y),
                bounds=self.bounds, tol=1e-10, 
                options={"maxiter":1000})
        self.sigma = res.x[0]
        self.H = res.x[1]
        return self
    
    def get_params(self, deep = False):
        return {'sigma':self.sigma}

#MULTIFRACTIONAL BLACK-SCHOLES
class MultiFractionalBlackScholes:
    def __init__(self, S=None, K=None, r=None, sigma=None, x0=[0.1, 0.5, 0.5], bounds=[(0.01, 0.9), (0.0, np.inf), (0.0, np.inf)], a=0.5, b=0.1):
        self.sigma = sigma
        self.r = r
        self.S = S
        self.K = K
        self.x0 = x0
        self.a = a
        self.b = b
        self.bounds=bounds

    def predict(self, X):
        return X.apply(lambda row: self.call(self.S, self.K, row['dte']/365.0, self.r, self.sigma, self.a, self.b), axis=1).tolist()

    def call(self, S, K, T, r, sigma, a, b):
        d1 = (np.log(S/K) + r*T + 0.5*(sigma**2)*T**(2*(a+b*T))) / (sigma*np.sqrt(T**(2*(a+b*T))))
        d2 = d1 - sigma * np.sqrt(T**(2*(a+b*T)))
        return S * norm.cdf(d1) - K * np.exp(-r*T)* norm.cdf(d2)

    def opt(self, x, X, y):
        candidate_prices = X.apply(lambda row: self.call(self.S, self.K, row['dte']/365.0, self.r, x[0], x[1], x[2]), axis=1).tolist()
        return np.linalg.norm(y - candidate_prices, 2)

    def fit(self, X, y, **kwargs):
        self.r = kwargs['r'][0]
        self.S = kwargs['S'][0]
        self.K = kwargs['K'][0]
        res = minimize(self.opt, method='SLSQP',  x0=self.x0, args=(X, y),
                bounds=self.bounds, tol=1e-10, 
                options={"maxiter":1000})
        self.sigma = res.x[0]
        self.a = res.x[1]
        self.b = res.x[2]
        return self
    
    def get_params(self, deep = False):
        return {'sigma':self.sigma}

# Market options data

In [13]:
# reading dataset
directory = 'spx_data/'

files = os.listdir(directory)
datasets = []

for file_path in files:
    if file_path == '.DS_Store':
        continue
    datasets.append(pd.read_csv(directory + file_path))

dataset = pd.concat(datasets).sort_values([' [QUOTE_DATE]', ' [STRIKE]'], ascending=True).reset_index(drop=True)
# Save dataset for further reading
dataset.to_csv('data.csv', index=False)

/var/folders/zb/11kb6dqn2dn65frk25grz2f80000gn/T/ipykernel_15740/1930122110.py:10: DtypeWarning: Columns (8,9,10,11,12,15,17,18,20,21,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets.append(pd.read_csv(directory + file_path))
/var/folders/zb/11kb6dqn2dn65frk25grz2f80000gn/T/ipykernel_15740/1930122110.py:10: DtypeWarning: Columns (8,9,10,11,12,15,17,18,20,21,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets.append(pd.read_csv(directory + file_path))
/var/folders/zb/11kb6dqn2dn65frk25grz2f80000gn/T/ipykernel_15740/1930122110.py:10: DtypeWarning: Columns (15,17,18,20,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets.append(pd.read_csv(directory + file_path))
/var/folders/zb/11kb6dqn2dn65frk25grz2f80000gn/T/ipykernel_15740/1930122110.py:10: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
 

# S&P 500 spot and risk-free rate data

In [14]:
# get annualized risk free rates based on 3-month us treasury bills rates
annualized_risk_free_rates = yf.download("^IRX", start='2000-01-01', end='2023-03-01', progress=False)
annualized_risk_free_rates.to_csv('risk_free_rates.csv')
annualized_risk_free_rates = annualized_risk_free_rates.reset_index()
#annualized_risk_free_rates['Date'] = annualized_risk_free_rates['Date'].dt.date

In [15]:
database=yf.download('^GSPC',start='2000-01-01', end='2023-03-01', progress=False)
database.to_csv('SP500_data.csv')

In [16]:
database.head(5)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000


# Simulation

In [ ]:
## Option data
option_prices = pd.read_csv('data.csv')
option_prices = option_prices.rename(columns=lambda x: x.strip().lower().replace('[', '').replace(']', ''))
option_prices = option_prices.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [9]:
option_prices.head(5)

,Unnamed: 0,quote_unixtime,quote_readtime,quote_date,quote_time_hours,underlying_last,expire_date,expire_unix,dte,c_delta,...,p_last,p_delta,p_gamma,p_vega,p_theta,p_rho,p_iv,p_volume,strike_distance,strike_distance_pct
0,6175750,1546462800,2019-01-02 16:00,2019-01-02,16.0,2509.98,2019-01-25,1548450000,23.0,1.0,...,0.00,0.00000,0.0,0.00094,-0.00734,0.00000,3.60114,NaN,2410.0,0.96
1,6175751,1546462800,2019-01-02 16:00,2019-01-02,16.0,2509.98,2019-02-01,1549054800,30.0,1.0,...,0.00,-0.00007,0.0,0.00095,-0.00528,0.00000,3.16895,NaN,2410.0,0.96
2,6175752,1546462800,2019-01-02 16:00,2019-01-02,16.0,2509.98,2019-12-20,1576875600,352.0,1.0,...,0.05,-0.00029,0.0,0.00365,-0.00012,-0.00077,0.93487,NaN,2410.0,0.96
3,6175753,1546462800,2019-01-02 16:00,2019-01-02,16.0,2509.98,2020-12-18,1608325200,716.0,1.0,...,0.10,0.00000,0.0,0.01725,-0.00132,-0.00776,0.72314,100.0,2410.0,0.96
4,6175754,1546462800,2019-01-02 16:00,2019-01-02,16.0,2509.98,2019-01-25,1548450000,23.0,1.0,...,0.00,-0.00046,0.0,0.00119,-0.00700,-0.00021,2.80167,NaN,2310.0,0.92


In [10]:
cores = 16
start='2020-04-01'
end='2022-12-31'
ticker='^GSPC'

# get dates range to test
index_days=yf.download(ticker,start=start, end=end, progress=False).index.strftime('%Y-%m-%d').unique()

result = pd.DataFrame(columns=['option_type', 'model_type', 'quote_date', 'strike', 'dte', 'price', 'underlying_price'])

#Loop for generating the results
for i in tqdm(range(index_days.shape[0])):
    cur_date = index_days[i]
    #Database is downloaded from yahoo finance and lag of returns defined
    option_prices_cur = option_prices.loc[(option_prices['quote_date'] == cur_date) & (option_prices['dte'] != 0.0),]
    strikes = option_prices_cur['strike'].unique().tolist()
    #skip if there are no option prices available to compare with
    if len(option_prices_cur) == 0:
        continue
    option_prices_calls = option_prices_cur.loc[option_prices_cur['c_volume']!='',]
    option_prices_puts = option_prices_cur.loc[option_prices_cur['p_volume']!='',]
    
    database=yf.download(ticker, start=pd.to_datetime(cur_date)-timedelta(days=300), end=cur_date, progress=False)
    r = annualized_risk_free_rates.loc[(annualized_risk_free_rates['Date'] == cur_date), 'Adj Close'].iloc[-1]/100.0 # risk free rate
    S = option_prices_cur['underlying_last'].iloc[0]
    
    for strike in strikes:
        op_cur_strike = option_prices_cur.loc[(option_prices_cur['strike'] == strike) & (option_prices_cur['c_last'] != 0.0),]
        if len(op_cur_strike) < 30:
            continue
        print(len(op_cur_strike))
        
        # try different K for K-FOLD cross validation
        if len(op_cur_strike) % 5 == 0:
            cv = 5
        elif len(op_cur_strike) % 4 == 0:
            cv = 4
        elif len(op_cur_strike) % 3 == 0:
            cv = 3
        elif len(op_cur_strike) % 2 == 0:
            cv = 2
        else:
            cv = len(op_cur_strike)
        # REAL
        real_prices = copy.copy(op_cur_strike)
        real_prices['option_type'] = 'call'
        real_prices['model_type'] = 'real'
        real_prices['price'] = op_cur_strike['c_last']#(real_prices['c_ask'] + real_prices['c_bid'])/2.0
        real_prices = real_prices[['option_type', 'model_type', 'price', 'quote_date', 'dte', 'underlying_last', 'strike']].rename(columns={'underlying_last': 'underlying_price'})
        result = result.append(real_prices)

        # BS MODEL
        y = op_cur_strike['c_last']
        bs_model = BlackScholes(S, strike, r)
        opt_prices = cross_val_predict(bs_model, op_cur_strike, op_cur_strike['c_last'], cv=cv, 
                                       fit_params={'r': [r], 'S': [S], 'K': [strike]}, n_jobs=cores)
        bs_prices = copy.copy(op_cur_strike)
        bs_prices['option_type'] = 'call'
        bs_prices['model_type'] = 'bs'
        bs_prices['price'] = opt_prices
        bs_prices = bs_prices[['option_type', 'model_type', 'price', 'quote_date', 'dte', 'underlying_last', 'strike']].rename(columns={'underlying_last': 'underlying_price'})
        result = result.append(bs_prices)

        # Fractional BS Model
        fbs_model = FractionalBlackScholes(S, strike, r)
        opt_prices = cross_val_predict(fbs_model, op_cur_strike, op_cur_strike['c_last'], cv=cv, 
                                       fit_params={'r': [r], 'S': [S], 'K': [strike]}, n_jobs=cores)
        fbs_prices = copy.copy(op_cur_strike)
        fbs_prices['option_type'] = 'call'
        fbs_prices['model_type'] = 'fbs'
        fbs_prices['price'] = opt_prices
        fbs_prices = fbs_prices[['option_type', 'model_type', 'price', 'quote_date', 'dte', 'underlying_last', 'strike']].rename(columns={'underlying_last': 'underlying_price'})
        result = result.append(fbs_prices)

        # Multifractional BS Model
        mbs_model = MultiFractionalBlackScholes(S, strike, r)
        opt_prices = cross_val_predict(mbs_model, op_cur_strike, op_cur_strike['c_last'], cv=cv, 
                                       fit_params={'r': [r], 'S': [S], 'K': [strike]}, n_jobs=cores)
        mbs_prices = copy.copy(op_cur_strike)
        mbs_prices['option_type'] = 'call'
        mbs_prices['model_type'] = 'mbs'
        mbs_prices['price'] = opt_prices
        mbs_prices = mbs_prices[['option_type', 'model_type', 'price', 'quote_date', 'dte', 'underlying_last', 'strike']].rename(columns={'underlying_last': 'underlying_price'})
        result = result.append(mbs_prices)

        # Merton model
        merton_model = Merton(S, strike, r)
        opt_prices = cross_val_predict(merton_model, op_cur_strike, op_cur_strike['c_last'], cv=cv, 
                                       fit_params={'r': [r], 'S': [S], 'K': [strike]}, n_jobs=cores)
        merton_prices = copy.copy(op_cur_strike)
        merton_prices['option_type'] = 'call'
        merton_prices['model_type'] = 'merton'
        merton_prices['price'] = opt_prices
        merton_prices = merton_prices[['option_type', 'model_type', 'price', 'quote_date', 'dte', 'underlying_last', 'strike']].rename(columns={'underlying_last': 'underlying_price'})
        result = result.append(merton_prices)

    if i % 20 == 0:
        result.to_csv('resultat2.csv')

30
30
33
30
32
31
34
32
34
30
34
31
34
32
35
33
34
33
33
34
35
33
34
31
34
34
31
34
30
31
32
30
31
30
31
30
33
30
32
32
34
34
35
31
35
32
35
32
35
34
35
33
33
34
35
33
34
31
34
34
31
33
30
31
32
30
31
30
30
30
33
30
32
32
33
33
34
32
35
31
34
31
36
35
35
32
34
34
36
33
34
30
36
33
30
33
30
31
30
32
31
30
32
31
33
33
34
31
35
32
35
34
34
33
33
33
35
32
34
30
35
33
33
31
30
30
33
30
31
31
32
31
33
33
34
31
35
32
35
34
34
34
34
33
36
33
34
31
35
34
35
32
31
30
33
30
30
30
30
31
30
32
32
33
30
35
32
34
33
33
33
34
33
35
32
34
30
35
33
35
33
30
30
32
30
31
32
31
32
34
31
33
33
33
31
33
34
36
33
35
34
34
36
35
33
31
31
30
30
31
30
31
33
31
32
32
32
31
33
34
35
34
35
32
34
35
31
35
31
34
33
30
31
30
30
31
30
31
33
31
33
32
33
31
33
32
34
34
35
33
36
32
35
35
36
32
35
35
31
35
33
30
30
30
30
32
30
33
32
32
30
34
31
33
34
35
32
35
32
34
33
35
32
33
32
35
31
34
33
31
30
30
30
31
31
34
30
34
32
32
30
34
31
35
34
35
33
35
33
34
34
35
32
33
33
35
34
33
33
31
30
30
30
33
32
31
31
32
32
34
32
35
34
3

100%|██████████| 694/694 [31:38:15<00:00, 164.11s/it]


In [14]:
result.to_csv('resultat2.csv')

,option_type,model_type,quote_date,strike,dte,price,underlying_price
1456078,call,real,2020-01-02,3050.0,1.00,192.470000,3258.14
1456079,call,real,2020-01-02,3050.0,4.00,170.320000,3258.14
1456080,call,real,2020-01-02,3050.0,6.00,178.870000,3258.14
1456081,call,real,2020-01-02,3050.0,8.00,197.000000,3258.14
1456082,call,real,2020-01-02,3050.0,11.00,172.520000,3258.14
...,...,...,...,...,...,...,...
1465274,call,mbs,2020-01-03,3500.0,363.00,50.776567,3234.35
1465275,call,mbs,2020-01-03,3500.0,378.00,53.497438,3234.35
1465276,call,mbs,2020-01-03,3500.0,440.96,64.842532,3234.35
1465277,call,mbs,2020-01-03,3500.0,531.96,80.972513,3234.35


# Performance evaluation

In [449]:
res = pd.read_csv('resultat.csv').drop(columns=['Unnamed: 0'])
res2 = pd.read_csv('resultat2.csv').drop(columns=['Unnamed: 0'])
res = pd.concat([res, res2])

In [450]:
res.head(5)

,option_type,model_type,quote_date,strike,dte,price,underlying_price
0,call,real,2020-01-02,3050.0,1.0,192.47,3258.14
1,call,real,2020-01-02,3050.0,4.0,170.32,3258.14
2,call,real,2020-01-02,3050.0,6.0,178.87,3258.14
3,call,real,2020-01-02,3050.0,8.0,197.00,3258.14
4,call,real,2020-01-02,3050.0,11.0,172.52,3258.14


In [498]:
comp = pd.pivot_table(res, values=['price'], index=['option_type', 'quote_date', 'strike', 'dte', 'underlying_price'],
                       columns=['model_type']).reset_index()

models = ['bs', 'fbs', 'mbs', 'merton']

for model in models:
    comp['mape_' + model] = np.abs(comp['price']['real'] - comp['price'][model])/(comp['price']['real'])*100.0
    comp['mae_' + model] = np.abs(comp['price']['real'] - comp['price'][model])

column_names = []
for col in comp.columns:
    if col[0] != 'price':
        column_names.append(col[0])
    else:
        column_names.append(col[1])

comp.columns = comp.columns.droplevel(0)
comp.columns = column_names

comp.head(10)

,option_type,quote_date,strike,dte,underlying_price,bs,fbs,mbs,merton,real,mape_bs,mae_bs,mape_fbs,mae_fbs,mape_mbs,mae_mbs,mape_merton,mae_merton
0,call,2020-01-02,3050.0,1.0,3258.14,208.264922,208.264922,208.264922,208.339553,192.47,8.206433,15.794922,8.206433,15.794922,8.206433,15.794922,8.245208,15.869553
1,call,2020-01-02,3050.0,4.0,3258.14,208.639658,208.639658,208.639658,208.938126,170.32,22.498625,38.319658,22.498625,38.319658,22.498625,38.319658,22.673864,38.618126
2,call,2020-01-02,3050.0,6.0,3258.14,208.889456,208.889456,208.889456,209.337103,178.87,16.782834,30.019456,16.782834,30.019456,16.782834,30.019456,17.033099,30.467103
3,call,2020-01-02,3050.0,8.0,3258.14,209.139234,209.139234,209.139234,209.736024,197.00,6.162047,12.139234,6.162047,12.139234,6.162047,12.139234,6.464987,12.736024
4,call,2020-01-02,3050.0,11.0,3258.14,209.513862,209.513862,209.513862,210.334298,172.52,21.443231,36.993862,21.443231,36.993862,21.443231,36.993862,21.918791,37.814298
5,call,2020-01-02,3050.0,13.0,3258.14,209.763594,209.763588,209.763588,210.733077,174.17,20.436122,35.593594,20.436119,35.593588,20.436119,35.593588,20.992752,36.563077
6,call,2020-01-02,3050.0,15.0,3258.14,210.013328,210.013294,210.013294,211.131798,200.83,4.572688,9.183328,4.572671,9.183294,4.572671,9.183294,5.129611,10.301798
7,call,2020-01-02,3050.0,22.0,3258.14,210.888437,210.887105,210.887105,212.526876,185.67,13.582397,25.218437,13.581680,25.217105,13.581680,25.217105,14.464844,26.856876
8,call,2020-01-02,3050.0,29.0,3258.14,211.770420,211.760683,211.760664,213.921258,187.27,13.082939,24.500420,13.077740,24.490683,13.077730,24.490664,14.231462,26.651258
9,call,2020-01-02,3050.0,32.0,3258.14,212.152832,212.135046,212.134970,214.518638,206.45,2.762331,5.702832,2.753716,5.685046,2.753679,5.684970,3.908277,8.068638


In [504]:
mape = pd.melt(comp, id_vars=['option_type', 'quote_date', 'strike', 'dte', 'underlying_price', 'bs', 'fbs', 'mbs', 'merton'], value_vars=['mape_bs', 'mape_fbs', 'mape_mbs', 'mape_merton']).drop(columns=['bs', 'fbs', 'mbs', 'merton']).rename(columns={'variable': 'mape'})
mape['dte_category'] = mape['dte'].apply(lambda x: 'DTE<30' if x < 30.0 else ('30<=DTE<120' if x >= 30.0 and x < 120 else 'DTE>=120'))
mape['mape_strike'] = mape['mape'] + mape['strike'].astype(str)
mape['month'] = pd.to_datetime(mape['quote_date']).dt.strftime('%Y')

mape = mape.groupby(['quote_date', 'dte_category', 'mape']).mean()['value'].reset_index()
mape['cat'] = mape['dte_category'] + '_' + mape['mape']
mape['cat'] = mape['cat'].replace('_mape', '')
mape['mape'] = mape['mape'].str.replace('mape_', '')
mape = mape.rename(columns={'mape':'model'})
mape['quote_date'] = pd.to_datetime(mape['quote_date'])
mape['month'] = mape['quote_date'].dt.strftime('%b')
mape['year'] = mape['quote_date'].dt.strftime('%Y')

mape.head(10)

<ipython-input-504-d7f65db048ef>:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mape = mape.groupby(['quote_date', 'dte_category', 'mape']).mean()['value'].reset_index()


,quote_date,dte_category,model,value,cat,month,year
0,2020-01-02,30<=DTE<120,bs,53.560428,30<=DTE<120_mape_bs,Jan,2020
1,2020-01-02,30<=DTE<120,fbs,17.454344,30<=DTE<120_mape_fbs,Jan,2020
2,2020-01-02,30<=DTE<120,mbs,16.044013,30<=DTE<120_mape_mbs,Jan,2020
3,2020-01-02,30<=DTE<120,merton,37.511110,30<=DTE<120_mape_merton,Jan,2020
4,2020-01-02,DTE<30,bs,141.235070,DTE<30_mape_bs,Jan,2020
5,2020-01-02,DTE<30,fbs,42.484581,DTE<30_mape_fbs,Jan,2020
6,2020-01-02,DTE<30,mbs,39.861000,DTE<30_mape_mbs,Jan,2020
7,2020-01-02,DTE<30,merton,98.422533,DTE<30_mape_merton,Jan,2020
8,2020-01-02,DTE>=120,bs,17.507269,DTE>=120_mape_bs,Jan,2020
9,2020-01-02,DTE>=120,fbs,17.145989,DTE>=120_mape_fbs,Jan,2020


In [505]:
# MEDIAN MAPE over years
median_mape = pd.DataFrame(mape.groupby(['year', 'dte_category', 'model']).quantile()['value'])
median_mape.reset_index()

<ipython-input-505-1992cf3611b5>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.quantile is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  median_mape = pd.DataFrame(mape.groupby(['year', 'dte_category', 'model']).quantile()['value'])


,year,dte_category,model,value
0,2020,30<=DTE<120,bs,21.523610
1,2020,30<=DTE<120,fbs,25.595293
2,2020,30<=DTE<120,mbs,28.639528
3,2020,30<=DTE<120,merton,21.837968
4,2020,DTE<30,bs,52.384500
5,2020,DTE<30,fbs,77.328274
6,2020,DTE<30,mbs,88.126224
7,2020,DTE<30,merton,47.165486
8,2020,DTE>=120,bs,15.673014
9,2020,DTE>=120,fbs,17.797769


In [463]:
class ggmatrix():
    def __init__(self, plot_width, plot_height):
        self.bunch = GGBunch()
        self.plot_width = plot_width
        self.plot_height = plot_height
        
    def add(self, col, row, spec):
        self.bunch.add_plot(spec, col * self.plot_width, row * self.plot_height, self.plot_width, self.plot_height)
        
    def show(self):
        self.bunch.show()

def time_plot(title, y_title, data):
    return ggplot() + geom_line(aes(x="quote_date", y="value", color="model"), data=data, sampling="none") + ggtitle(title)  + xlab("Quote date") + ylab(y_title) + scale_x_datetime(breaks=data[(data['quote_date'].dt.day == 1) | (data['quote_date'].dt.day == 2) | (data['quote_date'].dt.day == 3) | (data['quote_date'].dt.day == 4)]['quote_date'], format="%b %Y") + scale_y_log10() + theme(legend_position="none", axis_title_x='blank', axis_title_y='blank')

def box_plot(title, y_title, data):
    return ggplot() + \
    geom_boxplot(aes(x="model", y="value", \
                     fill=as_discrete("model")), \
                 data=data, size=0.8, alpha=.9) + \
    scale_x_discrete(name="model") + \
    ggtitle(title) + xlab("Model") + ylab(y_title) + \
    theme(legend_position='bottom', panel_grid='blank') + scale_y_log10()    

def time_box_plot(title, y_title, data):
    return ggplot() + \
    geom_boxplot(aes(x="month", y="value", fill=as_discrete("model")), \
                 data=data, size=.1, alpha=.9) + \
    facet_grid(x="year") + \
    ggtitle(title) + ylab(y_title) + ylim(8,250) + \
    theme(legend_position='none', panel_grid='blank') + scale_y_log10()

In [482]:
mape_30 = mape.loc[mape['dte_category'] == 'DTE<30',]
mape_60 = mape.loc[mape['dte_category'] == '30<=DTE<120',]
mape_120 = mape.loc[mape['dte_category'] == 'DTE>=120',]

matrix = ggmatrix(600, 800)
matrix.add(0, 0, box_plot("MAPE(%), DTEs < 30", "MAPE, %", mape_30))
matrix.add(1, 0, box_plot("MAPE(%), 30<=DTEs<120", "MAPE, %", mape_60))
matrix.add(2, 0, box_plot("MAPE(%), DTEs>=120", "MAPE, %", mape_120))

matrix.show()

In [464]:
matrix = ggmatrix(1000, 400)
matrix.add(0, 0, time_box_plot("MAPE(%) of Black-Scholes by months", "MAPE, %", mean_mape.loc[mean_mape['model'] == 'bs']))
matrix.add(0, 1, time_box_plot("MAPE(%) of Fractional Black-Scholes by months", "MAPE, %", mean_mape.loc[mean_mape['model'] == 
                                                                                                         'fbs']))
matrix.add(0, 2, time_box_plot("MAPE(%) of Multifractional Black-Scholes by months", "MAPE, %", mean_mape.loc[mean_mape['model'] == 'mbs']))
matrix.add(0, 3, time_box_plot("MAPE(%) of Merton model by months", "MAPE, %", mean_mape.loc[mean_mape['model'] == 'merton']))
matrix.show()

In [430]:
matrix = ggmatrix(1000, 800)
matrix.add(0, 0, time_plot("MAPE errors in 2020, DTE<30", "MAPE, %", mape_30.loc[(mape_30['quote_date'] >= '2020-01-01') & (mape_30['quote_date'] < '2021-01-01'),]))
matrix.add(1, 0, time_plot("MAPE errors in 2020, 30<=DTE<120", "MAPE, %", mape_60.loc[(mape_60['quote_date'] >= '2020-01-01') & (mape_60['quote_date'] < '2021-01-01'),]))
matrix.add(2, 0, time_plot("MAPE errors in 2020, DTE>=120", "MAPE, %", mape_120.loc[(mape_120['quote_date'] >= '2020-01-01') & (mape_120['quote_date'] < '2021-01-01'),]))
matrix.add(0, 1, time_plot("MAPE errors in 2021, DTE<30", "MAPE, %", mape_30.loc[(mape_30['quote_date'] >= '2021-01-01') & (mape_30['quote_date'] < '2022-01-01'),]))
matrix.add(1, 1, time_plot("MAPE errors in 2021, 30<=DTE<120", "MAPE, %", mape_60.loc[(mape_60['quote_date'] >= '2021-01-01') & (mape_60['quote_date'] < '2022-01-01'),]))
matrix.add(2, 1, time_plot("MAPE errors in 2021, DTE>=120", "MAPE, %", mape_120.loc[(mape_120['quote_date'] >= '2021-01-01') & (mape_120['quote_date'] < '2022-01-01'),]))
matrix.add(0, 2, time_plot("MAPE errors in 2022, DTE<30", "MAPE, %", mape_30.loc[(mape_30['quote_date'] >= '2022-01-01') & (mape_30['quote_date'] < '2023-01-01'),]))
matrix.add(1, 2, time_plot("MAPE errors in 2022, 30<=DTE<120", "MAPE, %", mape_60.loc[(mape_60['quote_date'] >= '2022-01-01') & (mape_60['quote_date'] < '2023-01-01'),]))
matrix.add(2, 2, time_plot("MAPE errors in 2022, DTE>=120", "MAPE, %", mape_120.loc[(mape_120['quote_date'] >= '2022-01-01') & (mape_120['quote_date'] < '2023-01-01'),]))
matrix.show()

In [228]:
from lets_plot import * 
ggplot() + \
geom_line(aes(x="quote_date", y="value", color="model"), data=mape_60, sampling="none" if mape_60.size < 2500 else sampling_systematic(n=2500)) + \
ggtitle("MAPE errors by days, 30<=DTE<120")  +\
xlab("Quote date") +\
ylab("MAPE, %") +\
 scale_y_log10()

In [70]:
from lets_plot import * 
ggplot() + \
geom_line(aes(x="quote_date", y="value", color="model"), data=mape_120, sampling="none" if mape_120.size < 2500 else sampling_systematic(n=2500)) + \
ggtitle("MAPE errors by days, DTE>=120")  +\
xlab("Quote date") +\
ylab("MAPE, %") +\
 scale_y_log10()

In [71]:
from lets_plot import * 
ggplot() + \
geom_line(aes(x="quote_date", y="value", color="model"), data=mape_120, sampling="none" if mape_120.size < 2500 else sampling_systematic(n=2500)) + \
ggtitle("MAPE errors by days, DTE>=120")  +\
xlab("Quote date") +\
ylab("MAPE, %") +\
 scale_y_log10()

In [499]:
mae = pd.melt(comp, id_vars=['option_type', 'quote_date', 'strike', 'dte', 'underlying_price', 'bs', 'fbs', 'mbs', 'merton'], value_vars=['mae_bs', 'mae_fbs', 'mae_mbs', 'mae_merton']).drop(columns=['bs', 'fbs', 'mbs', 'merton']).rename(columns={'variable': 'mae'})
mae['dte_category'] = mae['dte'].apply(lambda x: 'DTE<30' if x < 30.0 else ('30<=DTE<120' if x >= 30.0 and x < 120 else 'DTE>=120'))

mae = mae.groupby(['quote_date', 'dte_category', 'mae']).mean()['value'].reset_index()
mae['mae'] = mae['mae'].str.replace('mae_', '')
mae = mae.rename(columns={'mae':'model'})
mae['quote_date'] = pd.to_datetime(mae['quote_date'])
mae['month'] = mae['quote_date'].dt.strftime('%b')
mae['year'] = mae['quote_date'].dt.strftime('%Y')
mae.head(10)

<ipython-input-499-34da7489bd6c>:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mae = mae.groupby(['quote_date', 'dte_category', 'mae']).mean()['value'].reset_index()


,quote_date,dte_category,model,value,month,year
0,2020-01-02,30<=DTE<120,bs,12.002354,Jan,2020
1,2020-01-02,30<=DTE<120,fbs,8.056182,Jan,2020
2,2020-01-02,30<=DTE<120,mbs,8.823697,Jan,2020
3,2020-01-02,30<=DTE<120,merton,10.703829,Jan,2020
4,2020-01-02,DTE<30,bs,9.896056,Jan,2020
5,2020-01-02,DTE<30,fbs,7.085390,Jan,2020
6,2020-01-02,DTE<30,mbs,7.081817,Jan,2020
7,2020-01-02,DTE<30,merton,9.044745,Jan,2020
8,2020-01-02,DTE>=120,bs,25.180558,Jan,2020
9,2020-01-02,DTE>=120,fbs,30.910248,Jan,2020


In [506]:
median_mae = pd.DataFrame(mae.groupby(['year', 'dte_category', 'model']).quantile()['value'])
median_mae.reset_index()

<ipython-input-506-b586e9f10312>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.quantile is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  median_mae = pd.DataFrame(mae.groupby(['year', 'dte_category', 'model']).quantile()['value'])


,year,dte_category,model,value
0,2020,30<=DTE<120,bs,10.688648
1,2020,30<=DTE<120,fbs,11.025952
2,2020,30<=DTE<120,mbs,11.102656
3,2020,30<=DTE<120,merton,11.270666
4,2020,DTE<30,bs,5.713433
5,2020,DTE<30,fbs,8.398287
6,2020,DTE<30,mbs,8.680247
7,2020,DTE<30,merton,5.432122
8,2020,DTE>=120,bs,26.844290
9,2020,DTE>=120,fbs,35.056554


In [501]:
mae_30 = mae.loc[mae['dte_category'] == 'DTE<30',]
mae_60 = mae.loc[mae['dte_category'] == '30<=DTE<120',]
mae_120 = mae.loc[mae['dte_category'] == 'DTE>=120',]

In [502]:
matrix = ggmatrix(600, 800)
matrix.add(0, 0, box_plot("MAE, DTEs < 30", "ME", mae_30))
matrix.add(1, 0, box_plot("MAE, 30<=DTEs<120", "ME", mae_60))
matrix.add(2, 0, box_plot("MAE, DTEs>=120", "ME", mae_120))

matrix.show()

In [503]:
matrix = ggmatrix(1000, 800)
matrix.add(0, 0, time_plot("MAE errors in 2020, DTE<30", "MAE", mae_30.loc[(mae_30['quote_date'] >= '2020-01-01') & (mae_30['quote_date'] < '2021-01-01'),]))
matrix.add(1, 0, time_plot("MAE errors in 2020, 30<=DTE<120", "MAE", mae_60.loc[(mae_60['quote_date'] >= '2020-01-01') & (mae_60['quote_date'] < '2021-01-01'),]))
matrix.add(2, 0, time_plot("MAE errors in 2020, DTE>=120", "MAE", mae_120.loc[(mae_120['quote_date'] >= '2020-01-01') & (mae_120['quote_date'] < '2021-01-01'),]))
matrix.add(0, 1, time_plot("MAE errors in 2021, DTE<30", "MAE", mae_30.loc[(mae_30['quote_date'] >= '2021-01-01') & (mae_30['quote_date'] < '2022-01-01'),]))
matrix.add(1, 1, time_plot("MAE errors in 2021, 30<=DTE<120", "MAE", mae_60.loc[(mae_60['quote_date'] >= '2021-01-01') & (mae_60['quote_date'] < '2022-01-01'),]))
matrix.add(2, 1, time_plot("MAE errors in 2021, DTE>=120", "MAE", mae_120.loc[(mae_120['quote_date'] >= '2021-01-01') & (mae_120['quote_date'] < '2022-01-01'),]))
matrix.add(0, 2, time_plot("MAE errors in 2022, DTE<30", "MAE", mae_30.loc[(mae_30['quote_date'] >= '2022-01-01') & (mae_30['quote_date'] < '2023-01-01'),]))
matrix.add(1, 2, time_plot("MAE errors in 2022, 30<=DTE<120", "MAE", mae_60.loc[(mae_60['quote_date'] >= '2022-01-01') & (mae_60['quote_date'] < '2023-01-01'),]))
matrix.add(2, 2, time_plot("MAE errors in 2022, DTE>=120", "MAE", mae_120.loc[(mae_120['quote_date'] >= '2022-01-01') & (mae_120['quote_date'] < '2023-01-01'),]))
matrix.show()

In [209]:
ggplot() + \
geom_line(aes(x="quote_date", y="value", color="model"), data=mae_30, sampling="none") + \
ggtitle("MSE errors by days, DTE<30")  +\
xlab("Quote date") +\
ylab("MSE") +\
 scale_y_log10()

In [75]:
ggplot() + \
geom_line(aes(x="quote_date", y="value", color="model"), data=mse_60, sampling="none" if mape_60.size < 2500 else sampling_systematic(n=2500)) + \
ggtitle("MSE errors by days, 30<=DTE<120")  +\
xlab("Quote date") +\
ylab("MSE") +\
 scale_y_log10()

In [76]:
ggplot() + \
geom_line(aes(x="quote_date", y="value", color="model"), data=mse_120, sampling="none" if mape_60.size < 2500 else sampling_systematic(n=2500)) + \
ggtitle("MSE errors by days, DTE>=120")  +\
xlab("Quote date") +\
ylab("MSE") +\
 scale_y_log10()

In [24]:
single_day = res.loc[(res['quote_date'] == '2020-01-02') & (res['strike'] == 3250.0),]

In [25]:
from lets_plot import * 
ggplot() + \
geom_line(aes(x="dte", y="price", color="model_type"), data=single_day, sampling="none" if single_day.size < 2500 else sampling_systematic(n=2500)) + \
ggtitle("2020-01-02, Strike = 3250")  +\
xlab("DTE") +\
ylab("Call option price, $") +\
 scale_y_log10()

In [467]:
from lets_plot import * 
ggplot() + \
geom_line(aes(x="dte", y="price", color="model_type"), data=single_day, sampling="none") + \
ggtitle("2020-01-02, Strike = 3250")  +\
xlab("DTE") +\
ylab("Call option price, $") +\
 scale_y_log10()

In [216]:
single_day = res.loc[(res['quote_date'] == '2020-01-02') & (res['strike'] == 3100.0),]

In [217]:
from lets_plot import * 
ggplot() + \
geom_line(aes(x="dte", y="price", color="model_type"), data=single_day, sampling="none" if single_day.size < 2500 else sampling_systematic(n=2500)) + \
ggtitle("2020-01-02, Strike = 3100")  +\
xlab("DTE") +\
ylab("Call option price, $") +\
 scale_y_log10()

In [509]:
single_day = res.loc[(res['quote_date'] == '2020-01-02') & (res['strike'] == 3400.0) & (res['dte'] >= 30),]
ggplot() + \
geom_line(aes(x="dte", y="price", color="model_type"), data=single_day, sampling="none") + \
ggtitle("2020-01-02, Strike = 3400")  +\
xlab("DTE") +\
ylab("Call option price, $") +\
 scale_y_log10()